# General Manufacturing with OpenDXA

This tutorial demonstrates how to use OpenDXA for general manufacturing applications, focusing on production line optimization, quality control, and supply chain management.

## Prerequisites

- Understanding of OpenDXA basics (Workflow, Planning, and Reasoning layers)
- Familiarity with manufacturing concepts
- Python 3.8 or higher
- OpenDXA package installed

## Learning Objectives

By the end of this tutorial, you will be able to:

1. Create specialized workflows for general manufacturing
2. Implement custom reasoning strategies for production optimization
3. Design planning strategies for supply chain management
4. Integrate different layers for comprehensive manufacturing solutions

## 1. Setting Up the Environment

First, let's import the necessary components from OpenDXA.

In [ ]:
from opendxa.execution import (
    Workflow, WorkflowFactory, WorkflowExecutor,
    Plan, PlanFactory, PlanExecutor,
    Reasoning, ReasoningFactory, ReasoningExecutor,
    ExecutionContext
)
from opendxa.execution.workflow import WorkflowNode, WorkflowEdge
from opendxa.execution.planning import PlanStep, PlanEdge
from opendxa.execution.reasoning import BaseReasoningStrategy
from opendxa.common import NodeType

# Create an execution context
context = ExecutionContext()

## 2. Creating a Custom Reasoning Strategy

Let's create a specialized reasoning strategy for general manufacturing that can handle production optimization and quality control.

In [ ]:
class GeneralManufacturingStrategy(BaseReasoningStrategy):
    """Custom reasoning strategy for general manufacturing."""
    
    def __init__(self):
        super().__init__(
            name="general_manufacturing",
            description="Strategy for general manufacturing optimization"
        )
    
    def execute(self, task, context):
        """Execute the task using the general manufacturing strategy."""
        # Extract task details
        task_name = task.get("name")
        task_description = task.get("description")
        task_input = task.get("input", {})
        
        # Initialize result
        result = {
            "status": "SUCCESS",
            "output": {},
            "reasoning": []
        }
        
        # Execute task based on name
        if task_name == "analyze_production_line":
            result = self._analyze_production_line(task_input)
        elif task_name == "optimize_inventory":
            result = self._optimize_inventory(task_input)
        elif task_name == "quality_control":
            result = self._quality_control(task_input)
        else:
            result["status"] = "FAILED"
            result["reasoning"].append(f"Unknown task: {task_name}")
        
        return result
    
    def _analyze_production_line(self, input_data):
        """Analyze production line performance."""
        # Extract data
        production_data = input_data.get("production_data", [])
        
        # Initialize result
        result = {
            "status": "SUCCESS",
            "output": {
                "bottlenecks": [],
                "efficiency": 0.0,
                "recommendations": []
            },
            "reasoning": []
        }
        
        # Analyze production data
        if not production_data:
            result["status"] = "FAILED"
            result["reasoning"].append("No production data provided")
            return result
        
        # Calculate efficiency and identify bottlenecks
        total_time = 0
        idle_time = 0
        bottlenecks = []
        
        for station in production_data:
            station_name = station.get("name")
            processing_time = station.get("processing_time", 0)
            waiting_time = station.get("waiting_time", 0)
            
            total_time += processing_time + waiting_time
            idle_time += waiting_time
            
            if waiting_time > processing_time * 0.5:  # Arbitrary threshold
                bottlenecks.append({
                    "station": station_name,
                    "waiting_time": waiting_time,
                    "processing_time": processing_time
                })
        
        # Calculate efficiency
        efficiency = 1 - (idle_time / total_time) if total_time > 0 else 0
        
        # Generate recommendations
        recommendations = []
        if bottlenecks:
            recommendations.append("Address bottlenecks in the following stations:")
            for bottleneck in bottlenecks:
                recommendations.append(f"- {bottleneck['station']}: Reduce waiting time")
        
        if efficiency < 0.8:  # Arbitrary threshold
            recommendations.append("Improve overall production line efficiency")
        
        # Update result
        result["output"]["bottlenecks"] = bottlenecks
        result["output"]["efficiency"] = efficiency
        result["output"]["recommendations"] = recommendations
        result["reasoning"].append(f"Analyzed {len(production_data)} production stations")
        result["reasoning"].append(f"Identified {len(bottlenecks)} bottlenecks")
        result["reasoning"].append(f"Overall efficiency: {efficiency:.2%}")
        
        return result
    
    def _optimize_inventory(self, input_data):
        """Optimize inventory levels."""
        # Extract data
        inventory_data = input_data.get("inventory_data", [])
        
        # Initialize result
        result = {
            "status": "SUCCESS",
            "output": {
                "optimal_levels": {},
                "reorder_points": {},
                "recommendations": []
            },
            "reasoning": []
        }
        
        # Optimize inventory
        if not inventory_data:
            result["status"] = "FAILED"
            result["reasoning"].append("No inventory data provided")
            return result
        
        # Calculate optimal levels and reorder points
        for item in inventory_data:
            item_name = item.get("name")
            demand = item.get("demand", 0)
            lead_time = item.get("lead_time", 0)
            holding_cost = item.get("holding_cost", 0)
            
            # Simple EOQ calculation
            optimal_level = (2 * demand * lead_time) / holding_cost
            reorder_point = demand * lead_time
            
            # Update result
            result["output"]["optimal_levels"][item_name] = optimal_level
            result["output"]["reorder_points"][item_name] = reorder_point
            
            # Generate recommendations
            current_level = item.get("current_level", 0)
            if current_level < reorder_point:
                result["output"]["recommendations"].append(
                    f"Reorder {item_name}: Current level ({current_level}) below reorder point ({reorder_point})"
                )
            elif current_level > optimal_level * 1.2:  # 20% buffer
                result["output"]["recommendations"].append(
                    f"Reduce {item_name} inventory: Current level ({current_level}) above optimal ({optimal_level})"
                )
        
        result["reasoning"].append(f"Analyzed {len(inventory_data)} inventory items")
        result["reasoning"].append(f"Generated {len(result['output']['recommendations'])} recommendations")
        
        return result
    
    def _quality_control(self, input_data):
        """Perform quality control analysis."""
        # Extract data
        quality_data = input_data.get("quality_data", [])
        
        # Initialize result
        result = {
            "status": "SUCCESS",
            "output": {
                "defect_rate": 0.0,
                "quality_issues": [],
                "recommendations": []
            },
            "reasoning": []
        }
        
        # Analyze quality data
        if not quality_data:
            result["status"] = "FAILED"
            result["reasoning"].append("No quality data provided")
            return result
        
        # Calculate defect rate and identify issues
        total_units = 0
        defective_units = 0
        quality_issues = {}
        
        for batch in quality_data:
            batch_size = batch.get("size", 0)
            defects = batch.get("defects", [])
            
            total_units += batch_size
            defective_units += len(defects)
            
            for defect in defects:
                defect_type = defect.get("type")
                if defect_type in quality_issues:
                    quality_issues[defect_type] += 1
                else:
                    quality_issues[defect_type] = 1
        
        # Calculate defect rate
        defect_rate = defective_units / total_units if total_units > 0 else 0
        
        # Generate recommendations
        if defect_rate > 0.05:  # 5% threshold
            result["output"]["recommendations"].append("Implement additional quality control measures")
        
        for issue, count in quality_issues.items():
            if count > 10:  # Arbitrary threshold
                result["output"]["recommendations"].append(f"Investigate and address {issue} defects")
        
        # Update result
        result["output"]["defect_rate"] = defect_rate
        result["output"]["quality_issues"] = [
            {"type": issue, "count": count}
            for issue, count in quality_issues.items()
        ]
        result["reasoning"].append(f"Analyzed {total_units} units")
        result["reasoning"].append(f"Defect rate: {defect_rate:.2%}")
        result["reasoning"].append(f"Identified {len(quality_issues)} types of quality issues")
        
        return result

# Create an instance of the custom strategy
manufacturing_strategy = GeneralManufacturingStrategy()

# Register the strategy with the context
context.register_reasoning_strategy("GeneralManufacturingStrategy", manufacturing_strategy)

## 3. Creating a Manufacturing Workflow

Now, let's create a workflow for general manufacturing that includes production planning, quality control, and inventory management.

In [ ]:
# Create a general manufacturing workflow
manufacturing_workflow = Workflow(
    name="general_manufacturing",
    description="Workflow for general manufacturing process management",
    objective="Optimize manufacturing processes, ensure quality, and manage inventory effectively"
)

# Create nodes for the workflow
nodes = [
    WorkflowNode(name="start", node_type="START", description="Start of manufacturing workflow"),
    WorkflowNode(name="plan_production", node_type="TASK", description="Plan production schedule"),
    WorkflowNode(name="manage_inventory", node_type="TASK", description="Manage inventory levels"),
    WorkflowNode(name="execute_production", node_type="TASK", description="Execute production plan"),
    WorkflowNode(name="quality_control", node_type="TASK", description="Perform quality control"),
    WorkflowNode(name="analyze_results", node_type="TASK", description="Analyze production results"),
    WorkflowNode(name="optimize_process", node_type="TASK", description="Optimize manufacturing process"),
    WorkflowNode(name="update_plan", node_type="TASK", description="Update production plan"),
    WorkflowNode(name="end", node_type="END", description="End of manufacturing workflow")
]

# Add nodes to workflow
for node in nodes:
    manufacturing_workflow.add_node(node)

# Create edges to connect nodes
edges = [
    WorkflowEdge(nodes[0], nodes[1]),  # start -> plan_production
    WorkflowEdge(nodes[1], nodes[2]),  # plan_production -> manage_inventory
    WorkflowEdge(nodes[2], nodes[3]),  # manage_inventory -> execute_production
    WorkflowEdge(nodes[3], nodes[4]),  # execute_production -> quality_control
    WorkflowEdge(nodes[4], nodes[5]),  # quality_control -> analyze_results
    WorkflowEdge(nodes[5], nodes[6]),  # analyze_results -> optimize_process
    WorkflowEdge(nodes[6], nodes[7]),  # optimize_process -> update_plan
    WorkflowEdge(nodes[7], nodes[8])   # update_plan -> end
]

# Add edges to workflow
for edge in edges:
    manufacturing_workflow.add_edge(edge)

# Visualize the workflow
print("General Manufacturing Workflow:")
manufacturing_workflow.pretty_print()

## 4. Creating a Manufacturing Plan

Let's create a plan that breaks down the workflow into executable steps with specific reasoning strategies.

In [ ]:
# Create a plan based on the workflow
manufacturing_plan = Plan(
    name="general_manufacturing_plan",
    description="Plan for general manufacturing process management",
    objective="Optimize manufacturing processes, ensure quality, and manage inventory effectively"
)

# Create plan steps based on workflow nodes
plan_steps = [
    PlanStep(
        name="create_production_schedule",
        description="Create production schedule",
        expected_duration="1 day",
        reasoning_strategy="ChainOfThoughtStrategy"
    ),
    PlanStep(
        name="optimize_inventory_levels",
        description="Optimize inventory levels",
        expected_duration="1 day",
        reasoning_strategy="GeneralManufacturingStrategy"
    ),
    PlanStep(
        name="execute_production_plan",
        description="Execute production plan",
        expected_duration="5 days",
        reasoning_strategy="OODAStrategy"
    ),
    PlanStep(
        name="perform_quality_control",
        description="Perform quality control",
        expected_duration="2 days",
        reasoning_strategy="GeneralManufacturingStrategy"
    ),
    PlanStep(
        name="analyze_production_results",
        description="Analyze production results",
        expected_duration="1 day",
        reasoning_strategy="ChainOfThoughtStrategy"
    ),
    PlanStep(
        name="optimize_manufacturing_process",
        description="Optimize manufacturing process",
        expected_duration="2 days",
        reasoning_strategy="GeneralManufacturingStrategy"
    ),
    PlanStep(
        name="update_production_schedule",
        description="Update production schedule",
        expected_duration="1 day",
        reasoning_strategy="ChainOfThoughtStrategy"
    )
]

# Add steps to the plan
for step in plan_steps:
    manufacturing_plan.add_step(step)

# Create dependencies between plan steps
for i in range(len(plan_steps) - 1):
    manufacturing_plan.add_edge(PlanEdge(plan_steps[i], plan_steps[i + 1]))

# Visualize the plan
print("General Manufacturing Plan:")
manufacturing_plan.pretty_print()

## 5. Executing the Manufacturing Process

Now, let's execute the plan and see how it performs with sample manufacturing data.

In [ ]:
# Create sample manufacturing data
production_data = [
    {"name": "Assembly", "processing_time": 120, "waiting_time": 30},
    {"name": "Testing", "processing_time": 60, "waiting_time": 45},
    {"name": "Packaging", "processing_time": 30, "waiting_time": 15}
]

inventory_data = [
    {"name": "Raw Material A", "demand": 100, "lead_time": 5, "holding_cost": 2, "current_level": 50},
    {"name": "Component B", "demand": 50, "lead_time": 3, "holding_cost": 5, "current_level": 20},
    {"name": "Part C", "demand": 75, "lead_time": 4, "holding_cost": 3, "current_level": 30}
]

quality_data = [
    {"size": 100, "defects": [
        {"type": "Dimensional", "severity": "High"},
        {"type": "Surface", "severity": "Medium"}
    ]},
    {"size": 150, "defects": [
        {"type": "Functional", "severity": "High"},
        {"type": "Dimensional", "severity": "Low"}
    ]}
]

# Create a plan executor
executor = PlanExecutor()

# Execute the plan
result = executor.execute(manufacturing_plan, context)

# Print execution results
print("General Manufacturing Plan Execution Results:")
print(f"Status: {result.status}")
print(f"Completed Steps: {result.completed_steps}")
print(f"Failed Steps: {result.failed_steps}")
print(f"Total Duration: {result.total_duration}")

## 6. Analyzing Results

Let's analyze the results of our manufacturing process optimization.

In [ ]:
# Analyze production line
production_analysis = manufacturing_strategy.execute(
    {
        "name": "analyze_production_line",
        "description": "Analyze production line performance",
        "input": {"production_data": production_data}
    },
    context
)

print("\nProduction Line Analysis Results:")
print(f"Status: {production_analysis['status']}")
print(f"Overall Efficiency: {production_analysis['output']['efficiency']:.2%}")
print("\nBottlenecks:")
for bottleneck in production_analysis['output']['bottlenecks']:
    print(f"- {bottleneck['station']}: Waiting time = {bottleneck['waiting_time']} minutes")
print("\nRecommendations:")
for recommendation in production_analysis['output']['recommendations']:
    print(f"- {recommendation}")

# Optimize inventory
inventory_optimization = manufacturing_strategy.execute(
    {
        "name": "optimize_inventory",
        "description": "Optimize inventory levels",
        "input": {"inventory_data": inventory_data}
    },
    context
)

print("\nInventory Optimization Results:")
print(f"Status: {inventory_optimization['status']}")
print("\nOptimal Levels:")
for item, level in inventory_optimization['output']['optimal_levels'].items():
    print(f"- {item}: {level:.2f} units")
print("\nReorder Points:")
for item, point in inventory_optimization['output']['reorder_points'].items():
    print(f"- {item}: {point:.2f} units")
print("\nRecommendations:")
for recommendation in inventory_optimization['output']['recommendations']:
    print(f"- {recommendation}")

# Perform quality control
quality_analysis = manufacturing_strategy.execute(
    {
        "name": "quality_control",
        "description": "Perform quality control analysis",
        "input": {"quality_data": quality_data}
    },
    context
)

print("\nQuality Control Results:")
print(f"Status: {quality_analysis['status']}")
print(f"Defect Rate: {quality_analysis['output']['defect_rate']:.2%}")
print("\nQuality Issues:")
for issue in quality_analysis['output']['quality_issues']:
    print(f"- {issue['type']}: {issue['count']} occurrences")
print("\nRecommendations:")
for recommendation in quality_analysis['output']['recommendations']:
    print(f"- {recommendation}")

## Summary

In this tutorial, we've demonstrated how to use OpenDXA for general manufacturing applications:

1. Created a custom reasoning strategy for general manufacturing
2. Implemented a workflow for production management
3. Developed a plan with specific steps and reasoning strategies
4. Executed the plan with sample manufacturing data
5. Analyzed the results for production efficiency, inventory optimization, and quality control

The OpenDXA framework provides a powerful way to implement complex manufacturing workflows with:

- Custom reasoning strategies for production optimization
- Structured workflows for process management
- Detailed planning for execution coordination
- Comprehensive result analysis

## Next Steps

1. Explore more advanced manufacturing scenarios
2. Implement additional reasoning strategies
3. Add more sophisticated data analysis
4. Integrate with real manufacturing systems